In [1]:
import os
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms

from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

import numpy as np
from time import time
import copy

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

CUDA = torch.cuda.is_available()

In [2]:
RAW_DATA_DIR = "../Data/Raw/"
PROCESSED_DATA_DIR = "../Data/Processed/Regression/"

class CovariateDataset(Dataset):
    def __init__(self, file_name_pattern, file_name_args, train_size=0.8, test_size=0.2, test_train_complement=True):
        self.train = True
        self.test_on_all = False
        
        self.file_name = file_name_pattern.format(*file_name_args, "covar")
        self.assignment_file_name = file_name_pattern.format(*file_name_args, "assignment")
        
        self.data = np.loadtxt(RAW_DATA_DIR + self.file_name + ".csv", delimiter=",")[:, 1:] # remove bias
        self.assignment_data = np.loadtxt(
            RAW_DATA_DIR + self.assignment_file_name + ".csv", delimiter=",").astype(int)
        
        self.all_indeces = np.array(range(len(self.data)))
        treat_indeces = self.all_indeces[self.assignment_data.astype(int) == 1]
        control_indeces = self.all_indeces[self.assignment_data.astype(int) == 0]
        
        num_training = int(len(self.data)*train_size)
        
        self.train_indeces = np.random.choice(self.all_indeces, num_training, replace=False)
        if test_train_complement:
            self.test_indeces = list(set(self.all_indeces)^set(self.train_indeces))      
        else:
            self.test_indeces = np.random.choice(self.all_indeces, int(len(self.data)*(1-test_size)), replace=False)
        
        num_treated_in_train = len(np.intersect1d(treat_indeces, self.train_indeces, assume_unique=True))
        num_control_in_train = num_training - num_treated_in_train
        
        treat_weight = num_training / (2 * num_treated_in_train)
        control_weight = num_training / (2 * num_control_in_train)
        
        weighter = np.vectorize(lambda index: treat_weight if index in\
            treat_indeces else control_weight)
        
        self.weights = weighter(self.all_indeces)
        
    def active_data(self, index=0):
        if self.train:
            return self.data[self.train_indeces], self.assignment_data[self.train_indeces], \
                self.weights[self.train_indeces][index]
        else:
            if self.test_on_all:
                indeces = self.all_indeces
            else: 
                indeces = self.test_indeces
            
            return self.data[indeces], self.assignment_data[indeces], 1
            
    def __getitem__(self, index):
        covar_data, assignment_data, weight_data = self.active_data(index)
        class_vector = np.zeros(2)
        class_vector[int(assignment_data[index])] = 1
        
        return (covar_data[index], class_vector, weight_data)

    def __len__(self):
        return self.active_data()[0].shape[0]
    
    def save_processed_data(self, data):
        name = PROCESSED_DATA_DIR + self.file_name+".csv"
        np.savetxt(name, data, delimiter=",")
        
def get_datasets(file_name_format, file_name_args, **kwargs):
    train_set = CovariateDataset(file_name_format, file_name_args, **kwargs)
    test_set = copy.deepcopy(train_set)
    test_set.train = False

    predict_set = copy.deepcopy(train_set)
    predict_set.train = False
    predict_set.test_on_all = True
    
    return train_set, test_set, predict_set

In [3]:
# Based on an example from https://github.com/pytorch/examples/blob/master/vae/main.py
# Extended to place a different prior on binary vs normal vars

class Regressor(nn.Module):
    def __init__(self):
        super(Regressor, self).__init__()
        
        INTERMEDIATE_DIMS_1 = 16
        INTERMEDIATE_DIMS_2 = 16
        INTERMEDIATE_DIMS_3 = 16
        INTERMEDIATE_DIMS_4 = 16
#         INTERMEDIATE_DIMS_5 = 16
#         INTERMEDIATE_DIMS_6 = 8

        FEATURES = 10

        LOSS_SCALE = 1

        # ENCODER LAYERS
        self.dense1 = nn.Linear(FEATURES, INTERMEDIATE_DIMS_1)
        self.dense2 = nn.Linear(INTERMEDIATE_DIMS_1, INTERMEDIATE_DIMS_2)
        self.dense3 = nn.Linear(INTERMEDIATE_DIMS_2, INTERMEDIATE_DIMS_3)
        self.dense4 = nn.Linear(INTERMEDIATE_DIMS_3, INTERMEDIATE_DIMS_4)
#         self.dense5 = nn.Linear(INTERMEDIATE_DIMS_4, INTERMEDIATE_DIMS_5)
#         self.dense6 = nn.Linear(INTERMEDIATE_DIMS_5, INTERMEDIATE_DIMS_6)
        self.dense5 = nn.Linear(INTERMEDIATE_DIMS_4, 2)
        
        # Activations
        self.softmax = nn.Softmax(dim=1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.6)

    def forward(self, x):
        h1 = self.dropout(self.relu(self.dense1(x)))
        h2 = self.dropout(self.relu(self.dense2(h1)))
        h3 = self.dropout(self.relu(self.dense3(h2)))
        h4 = self.dropout(self.relu(self.dense4(h3)))
#         h5 = self.dropout(self.relu(self.dense5(h4)))
#         h6 = self.dropout(self.relu(self.dense6(h5)))
        
        return self.softmax(self.dense5(h4))



def train(model, optimizer, epoch, train_loader, log_results=False):
    model.train()
    train_loss = 0

    for batch_idx, (data, target_class, weights) in enumerate(train_loader):
        data = Variable(data)
        target_class = Variable(target_class)
        weights = Variable(weights)
        
        data = data.float()
        target_class = target_class.float()
        weights = weights.float()
        
        if CUDA:
            data = data.cuda()
            target_class = target_class.cuda()
            weights = weights.cuda()
        
        optimizer.zero_grad()

        output_propensity = model(data)
        
        # calculate loss
        loss_criterion = nn.BCELoss(weight=weights.view(weights.shape[0], 1), size_average=False)
        loss = loss_criterion(output_propensity, target_class)

        train_loss += loss.data[0]
        
        # Find the gradient and descend
        loss.backward()
        optimizer.step()
        
    if log_results:
        print('====> Epoch: {} Average loss: {:.8f}'.format(
              epoch, train_loss / len(train_loader.dataset)))
        
        
def test(model, epoch, test_loader):
    # toggle model to test / inference mode
    model.eval()
    test_loss = 0

    for i, (data, target_class, weights) in enumerate(test_loader):
        data = Variable(data, volatile=True)
        target_class = Variable(target_class, volatile=True)
        weights = Variable(weights, volatile=True)
        
        data = data.float()
        target_class = target_class.float()
        weights = weights.float()
        
        if CUDA:
            data = data.cuda()
            target_class = target_class.cuda()
            weights = weights.cuda()

        output_propensity = model(data)
        
        # calculate loss
        loss_criterion = nn.BCELoss(weight=weights.view(weights.shape[0], 1), size_average=False)
        loss = loss_criterion(output_propensity, target_class)
        test_loss += loss.data[0]

    test_loss /= len(test_loader.dataset)
    
    if CUDA:
        output_propensity = output_propensity.cpu()
        target_class = target_class.cpu()
        
    score = accuracy(output_propensity.data.numpy(), target_class.data.numpy(), verbose=False)
    print('====> Test set loss: {:.4f}, {}%'.format(test_loss, score*100))
    
def predict(model, predict_loader):
    # Show reconstruction
    model.eval()
    print("Training state: ", model.training)
    
    original_data, targets, _ = next(iter(predict_loader))
    
    original_data = Variable(original_data)
    original_data = original_data.float()
    
    if CUDA:
        original_data = original_data.cuda()
        
    return original_data, targets, model(original_data)

def accuracy(output_data, targets, verbose=True):
        
    classes = np.argmax(output_data, axis=1)
    targets = np.argmax(targets, axis=1)
    
    if verbose:
        print(classification_report(targets, classes))
    return accuracy_score(targets, classes)

In [4]:
def train_model(model_class, train_set, test_set, predict_set, dataset_number, verbose=True, model=None):
    if model is None:
        model = model_class()
        if CUDA:
            model = model.cuda()

    num_epochs = 750
    train_batch_size = 64
    test_batch_size = 250
    learning_rate = 1e-3
    lr_sched = True
         
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [int(num_epochs/5), int(num_epochs/2)], gamma=0.1)

    train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=test_batch_size, shuffle=True)
    predict_loader = DataLoader(predict_set, batch_size=1000, shuffle=False)
    
    for epoch in range(1, num_epochs+1):
        checkpoint_interval = int(num_epochs/10)
        
        if lr_sched:
            scheduler.step()

        log = False
        if epoch%checkpoint_interval == 0:
            log = True
            
        train(model, optimizer, epoch, train_loader, log_results=log)
        if log:
            test(model, epoch, test_loader)
    
    original_data, targets, output = predict(model, predict_loader)
    if CUDA:
        output = output.cpu()
        targets = targets.cpu()
    
    return model, original_data, targets, output

def encode_data(dataset, output_data):
    
    if CUDA:
        output_data = output_data.cpu()
        
    dataset.save_processed_data(output_data.data.numpy()[:, 1])


In [5]:
def run_logistic(train_set, verbose=True):
    model = LogisticRegression(class_weight="balanced")
    
    X = train_set.data
    y = train_set.assignment_data

    X_train = X[train_set.train_indeces]
    X_test = X[train_set.test_indeces]
    y_train = y[train_set.train_indeces]
    y_test = y[train_set.test_indeces]

    model.fit(X_train, y_train)
    predictions = model.predict(X)
    
    if verbose:
        print(classification_report(y, predictions))
    
    return accuracy_score(y, predictions)

In [6]:
train_set, test_set, predict_set = get_datasets(
    "n_{}_model_{}_v_{}_{}_data", [1000, "G_mod_nadd_mod_nlin", 1],
    train_size=0.8, test_train_complement=True)

start = time()
trained_model, original_data, targets, output = \
    train_model(Regressor, train_set, test_set,predict_set, 1,verbose=True)
print("Elapsed: ", time() - start)


acc = accuracy(output.data.cpu().numpy(), targets.numpy(), verbose=False)
print("Complete set accuracy: {}%".format(acc*100))

encode_data(train_set, output)

====> Epoch: 75 Average loss: 1.28134798
====> Test set loss: 1.2909, 62.0%
====> Epoch: 150 Average loss: 1.20173033
====> Test set loss: 1.2127, 69.0%
====> Epoch: 225 Average loss: 1.20760253
====> Test set loss: 1.2092, 69.0%
====> Epoch: 300 Average loss: 1.21259918
====> Test set loss: 1.2043, 69.0%
====> Epoch: 375 Average loss: 1.20176897
====> Test set loss: 1.2014, 70.0%
====> Epoch: 450 Average loss: 1.24033634
====> Test set loss: 1.2018, 69.5%
====> Epoch: 525 Average loss: 1.24107657
====> Test set loss: 1.1998, 70.5%
====> Epoch: 600 Average loss: 1.23011834
====> Test set loss: 1.1991, 70.5%
====> Epoch: 675 Average loss: 1.22299410
====> Test set loss: 1.1984, 70.5%
====> Epoch: 750 Average loss: 1.21179613
====> Test set loss: 1.1980, 70.5%
Training state:  False
Elapsed:  46.78293299674988
Complete set accuracy: 72.0%


### Runner

In [6]:
assignment_model_names = ['A_add_lin', 'B_add_mild_nlin', 'C_add_mod_nlin', 'D_mild_nadd_lin',
                     'E_mild_nadd_mild_nlin', 'F_mod_nadd_lin', 'G_mod_nadd_mod_nlin']

nn_accuracies = []
log_accuracies = []

for dataset_number in range(200, 250):
    print("Starting run for Dataset {}".format(dataset_number))
    
    for model_name in assignment_model_names:
        print("---- Running for model name: ", model_name)
        
        start = time()

        train_set, test_set, predict_set = get_datasets(
            "n_{}_model_{}_v_{}_{}_data", [1000, model_name, dataset_number], train_size=0.8)

        trained_model, original_data, targets, output = \
            train_model(Regressor, train_set, test_set,predict_set, 1,verbose=True)
        
        nn_acc = accuracy(output.data.cpu().numpy(), targets.numpy(), verbose=False)
        print("Complete set accuracy: {}%".format(nn_acc*100))
        
        log_acc = run_logistic(train_set, verbose=False)
        print("Log accuracy: {}%".format(log_acc*100))
        
        nn_accuracies.append(nn_acc)
        log_accuracies.append(log_acc)

        encode_data(train_set, output)

        print("---- Done in ", time() - start, " seconds\n")
                
    print("================\n\n")

Starting run for Dataset 200
---- Running for model name:  A_add_lin
====> Epoch: 75 Average loss: 1.26779034
====> Test set loss: 1.2575, 66.0%
====> Epoch: 150 Average loss: 1.22910618
====> Test set loss: 1.2447, 67.0%
====> Epoch: 225 Average loss: 1.25967963
====> Test set loss: 1.2332, 68.0%
====> Epoch: 300 Average loss: 1.19273018
====> Test set loss: 1.2298, 69.0%
====> Epoch: 375 Average loss: 1.22972327
====> Test set loss: 1.2304, 69.0%
====> Epoch: 450 Average loss: 1.19131581
====> Test set loss: 1.2281, 69.0%
====> Epoch: 525 Average loss: 1.15323077
====> Test set loss: 1.2266, 69.0%
====> Epoch: 600 Average loss: 1.21243670
====> Test set loss: 1.2268, 69.0%
====> Epoch: 675 Average loss: 1.20288382
====> Test set loss: 1.2268, 69.0%
====> Epoch: 750 Average loss: 1.18059282
====> Test set loss: 1.2258, 68.5%
Training state:  False
Complete set accuracy: 72.7%
Log accuracy: 71.3%
---- Done in  56.105324029922485  seconds

---- Running for model name:  B_add_mild_nlin
=

====> Epoch: 675 Average loss: 1.13613984
====> Test set loss: 1.1529, 71.0%
====> Epoch: 750 Average loss: 1.16026847
====> Test set loss: 1.1532, 71.0%
Training state:  False
Complete set accuracy: 73.5%
Log accuracy: 71.2%
---- Done in  50.29844784736633  seconds

---- Running for model name:  C_add_mod_nlin
====> Epoch: 75 Average loss: 1.27278162
====> Test set loss: 1.2862, 63.5%
====> Epoch: 150 Average loss: 1.26089411
====> Test set loss: 1.2374, 67.0%
====> Epoch: 225 Average loss: 1.18183851
====> Test set loss: 1.2189, 69.0%
====> Epoch: 300 Average loss: 1.20712589
====> Test set loss: 1.2186, 70.0%
====> Epoch: 375 Average loss: 1.21253093
====> Test set loss: 1.2187, 69.5%
====> Epoch: 450 Average loss: 1.20140708
====> Test set loss: 1.2179, 69.5%
====> Epoch: 525 Average loss: 1.16682948
====> Test set loss: 1.2169, 69.5%
====> Epoch: 600 Average loss: 1.17670638
====> Test set loss: 1.2169, 69.5%
====> Epoch: 675 Average loss: 1.17946185
====> Test set loss: 1.2164, 6

====> Epoch: 450 Average loss: 1.14010044
====> Test set loss: 1.2018, 69.0%
====> Epoch: 525 Average loss: 1.17546360
====> Test set loss: 1.2009, 69.0%
====> Epoch: 600 Average loss: 1.14980158
====> Test set loss: 1.2009, 68.5%
====> Epoch: 675 Average loss: 1.13112756
====> Test set loss: 1.1996, 68.5%
====> Epoch: 750 Average loss: 1.15935062
====> Test set loss: 1.2010, 68.5%
Training state:  False
Complete set accuracy: 74.2%
Log accuracy: 74.8%
---- Done in  51.3265380859375  seconds

---- Running for model name:  E_mild_nadd_mild_nlin
====> Epoch: 75 Average loss: 1.24785307
====> Test set loss: 1.1961, 72.0%
====> Epoch: 150 Average loss: 1.16563333
====> Test set loss: 1.1408, 71.5%
====> Epoch: 225 Average loss: 1.15876124
====> Test set loss: 1.1319, 73.0%
====> Epoch: 300 Average loss: 1.15197272
====> Test set loss: 1.1290, 72.0%
====> Epoch: 375 Average loss: 1.18061963
====> Test set loss: 1.1276, 72.0%
====> Epoch: 450 Average loss: 1.19201917
====> Test set loss: 1.1

====> Epoch: 225 Average loss: 1.22133017
====> Test set loss: 1.1427, 68.5%
====> Epoch: 300 Average loss: 1.20722067
====> Test set loss: 1.1397, 67.5%
====> Epoch: 375 Average loss: 1.17223588
====> Test set loss: 1.1370, 67.5%
====> Epoch: 450 Average loss: 1.21319200
====> Test set loss: 1.1373, 67.5%
====> Epoch: 525 Average loss: 1.17297616
====> Test set loss: 1.1375, 67.5%
====> Epoch: 600 Average loss: 1.16287488
====> Test set loss: 1.1379, 67.5%
====> Epoch: 675 Average loss: 1.17504154
====> Test set loss: 1.1376, 67.5%
====> Epoch: 750 Average loss: 1.19427286
====> Test set loss: 1.1378, 67.5%
Training state:  False
Complete set accuracy: 72.6%
Log accuracy: 70.8%
---- Done in  50.34458112716675  seconds

---- Running for model name:  G_mod_nadd_mod_nlin
====> Epoch: 75 Average loss: 1.29844201
====> Test set loss: 1.2388, 68.0%
====> Epoch: 150 Average loss: 1.24031983
====> Test set loss: 1.1776, 68.5%
====> Epoch: 225 Average loss: 1.19540387
====> Test set loss: 1.16

====> Epoch: 75 Average loss: 1.29541533
====> Test set loss: 1.1705, 75.0%
====> Epoch: 150 Average loss: 1.22989495
====> Test set loss: 1.1321, 72.5%
====> Epoch: 225 Average loss: 1.20670083
====> Test set loss: 1.1280, 73.0%
====> Epoch: 300 Average loss: 1.21351707
====> Test set loss: 1.1253, 73.0%
====> Epoch: 375 Average loss: 1.20797273
====> Test set loss: 1.1250, 72.5%
====> Epoch: 450 Average loss: 1.23586851
====> Test set loss: 1.1250, 72.5%
====> Epoch: 525 Average loss: 1.23705543
====> Test set loss: 1.1248, 72.0%
====> Epoch: 600 Average loss: 1.21036331
====> Test set loss: 1.1248, 72.0%
====> Epoch: 675 Average loss: 1.22828659
====> Test set loss: 1.1246, 72.0%
====> Epoch: 750 Average loss: 1.21080174
====> Test set loss: 1.1243, 72.0%
Training state:  False
Complete set accuracy: 71.8%
Log accuracy: 68.60000000000001%
---- Done in  50.69281601905823  seconds

---- Running for model name:  B_add_mild_nlin
====> Epoch: 75 Average loss: 1.29501413
====> Test set lo

...

====> Epoch: 750 Average loss: 1.20330127
====> Test set loss: 1.1578, 70.0%
Training state:  False
Complete set accuracy: 70.3%
Log accuracy: 71.7%
---- Done in  64.31988883018494  seconds

---- Running for model name:  G_mod_nadd_mod_nlin
====> Epoch: 75 Average loss: 1.27196277
====> Test set loss: 1.2087, 66.0%
====> Epoch: 150 Average loss: 1.22582780
====> Test set loss: 1.1595, 71.0%
====> Epoch: 225 Average loss: 1.16146627
====> Test set loss: 1.1607, 71.0%
====> Epoch: 300 Average loss: 1.17231604
====> Test set loss: 1.1554, 71.0%
====> Epoch: 375 Average loss: 1.18672940
====> Test set loss: 1.1592, 71.0%
====> Epoch: 450 Average loss: 1.20833646
====> Test set loss: 1.1581, 71.0%
====> Epoch: 525 Average loss: 1.14689657
====> Test set loss: 1.1574, 71.0%
====> Epoch: 600 Average loss: 1.27184329
====> Test set loss: 1.1573, 71.0%
====> Epoch: 675 Average loss: 1.19284444
====> Test set loss: 1.1579, 71.0%
====> Epoch: 750 Average loss: 1.18058105
====> Test set loss: 1.15

====> Epoch: 450 Average loss: 1.20911432
====> Test set loss: 1.1622, 68.0%
====> Epoch: 525 Average loss: 1.18176188
====> Test set loss: 1.1629, 68.0%
====> Epoch: 600 Average loss: 1.21204575
====> Test set loss: 1.1625, 68.0%
====> Epoch: 675 Average loss: 1.15039735
====> Test set loss: 1.1630, 68.0%
====> Epoch: 750 Average loss: 1.18387087
====> Test set loss: 1.1626, 68.0%
Training state:  False
Complete set accuracy: 73.5%
Log accuracy: 71.7%
---- Done in  64.69607186317444  seconds

---- Running for model name:  B_add_mild_nlin
====> Epoch: 75 Average loss: 1.27461136
====> Test set loss: 1.2577, 63.5%
====> Epoch: 150 Average loss: 1.24791983
====> Test set loss: 1.2177, 64.5%
====> Epoch: 225 Average loss: 1.22012060
====> Test set loss: 1.2145, 65.5%
====> Epoch: 300 Average loss: 1.23494573
====> Test set loss: 1.2177, 65.5%
====> Epoch: 375 Average loss: 1.25123439
====> Test set loss: 1.2177, 65.5%
====> Epoch: 450 Average loss: 1.20388921
====> Test set loss: 1.2170, 

====> Epoch: 225 Average loss: 1.18646655
====> Test set loss: 1.1903, 68.5%
====> Epoch: 300 Average loss: 1.17574126
====> Test set loss: 1.1855, 69.5%
====> Epoch: 375 Average loss: 1.19031587
====> Test set loss: 1.1897, 69.0%
====> Epoch: 450 Average loss: 1.19287600
====> Test set loss: 1.1882, 69.0%
====> Epoch: 525 Average loss: 1.18464190
====> Test set loss: 1.1868, 69.0%
====> Epoch: 600 Average loss: 1.18251055
====> Test set loss: 1.1871, 69.0%
====> Epoch: 675 Average loss: 1.19000759
====> Test set loss: 1.1858, 69.0%
====> Epoch: 750 Average loss: 1.19835670
====> Test set loss: 1.1850, 69.0%
Training state:  False
Complete set accuracy: 73.8%
Log accuracy: 69.0%
---- Done in  61.239063024520874  seconds

---- Running for model name:  D_mild_nadd_lin
====> Epoch: 75 Average loss: 1.23165563
====> Test set loss: 1.1341, 76.0%
====> Epoch: 150 Average loss: 1.19259408
====> Test set loss: 1.0947, 74.0%
====> Epoch: 225 Average loss: 1.14770451
====> Test set loss: 1.0947,

====> Epoch: 75 Average loss: 1.19782553
====> Test set loss: 1.1274, 73.5%
====> Epoch: 150 Average loss: 1.14363296
====> Test set loss: 1.0831, 74.0%
====> Epoch: 225 Average loss: 1.12480516
====> Test set loss: 1.0849, 74.0%
====> Epoch: 300 Average loss: 1.13503912
====> Test set loss: 1.0844, 73.5%
====> Epoch: 375 Average loss: 1.16903276
====> Test set loss: 1.0887, 73.0%
====> Epoch: 450 Average loss: 1.08387570
====> Test set loss: 1.0885, 73.0%
====> Epoch: 525 Average loss: 1.11477407
====> Test set loss: 1.0881, 73.0%
====> Epoch: 600 Average loss: 1.14211728
====> Test set loss: 1.0875, 73.0%
====> Epoch: 675 Average loss: 1.10794040
====> Test set loss: 1.0873, 73.0%
====> Epoch: 750 Average loss: 1.07754267
====> Test set loss: 1.0875, 73.0%
Training state:  False
Complete set accuracy: 77.0%
Log accuracy: 76.3%
---- Done in  66.45229983329773  seconds

---- Running for model name:  F_mod_nadd_lin
====> Epoch: 75 Average loss: 1.29008005
====> Test set loss: 1.2773, 58

====> Epoch: 750 Average loss: 1.22464006
====> Test set loss: 1.1409, 67.0%
Training state:  False
Complete set accuracy: 71.7%
Log accuracy: 70.8%
---- Done in  65.09491491317749  seconds

---- Running for model name:  G_mod_nadd_mod_nlin
====> Epoch: 75 Average loss: 1.26198279
====> Test set loss: 1.2591, 63.0%
====> Epoch: 150 Average loss: 1.14200326
====> Test set loss: 1.2100, 65.0%
====> Epoch: 225 Average loss: 1.17662428
====> Test set loss: 1.2186, 64.0%
====> Epoch: 300 Average loss: 1.15023917
====> Test set loss: 1.2147, 63.5%
====> Epoch: 375 Average loss: 1.18028642
====> Test set loss: 1.2117, 64.0%
====> Epoch: 450 Average loss: 1.17352239
====> Test set loss: 1.2117, 64.0%
====> Epoch: 525 Average loss: 1.15895089
====> Test set loss: 1.2128, 64.0%
====> Epoch: 600 Average loss: 1.19376178
====> Test set loss: 1.2128, 64.0%
====> Epoch: 675 Average loss: 1.17144629
====> Test set loss: 1.2130, 64.0%
====> Epoch: 750 Average loss: 1.13143662
====> Test set loss: 1.21